In [ ]:
import os
import cv2
import numpy as np

# Define paths to input and output folders
input_folder = '/content/Live Phase II/left and right views'
output_folder = '/content/Live Phase II/Cyclopean Image '

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to compute gradients for a HSV image
def compute_gradients(image):
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return grad_x, grad_y

# Function to compute the largest eigenvector of a 3x3 matrix at each pixel
def compute_disparity_map(S11, S12, S13, S22, S23, S33):
    disparity_map = np.zeros(S11.shape)
    for x in range(S11.shape[0]):
        for y in range(S11.shape[1]):
            # Construct matrix S
            S = np.array([[S11[x, y], S12[x, y], S13[x, y]],
                          [S12[x, y], S22[x, y], S23[x, y]],
                          [S13[x, y], S23[x, y], S33[x, y]]])

            # Compute eigenvalues and eigenvectors
            eigenvalues, eigenvectors = np.linalg.eigh(S)

            # Find the largest eigenvalue and corresponding eigenvector
            largest_eigenvalue_index = np.argmax(eigenvalues)
            largest_eigenvector = eigenvectors[:, largest_eigenvalue_index]

            # Disparity value is the magnitude of the largest eigenvector
            disparity_map[x, y] = np.linalg.norm(largest_eigenvector)
    return disparity_map

# Function to create a cyclopean image
def create_cyclopean_image(left_image, right_image, disparity_map, window_size=9):
    height, width, _ = left_image.shape
    cyclopean_image = np.zeros_like(left_image, dtype=np.float64)
    half_window = window_size // 2

    for x in range(half_window, height - half_window):
        for y in range(half_window, width - half_window):
            disparity = int(disparity_map[x, y])
            right_y = min(max(y - disparity, half_window), width - half_window - 1)

            left_patch = left_image[x - half_window:x + half_window + 1, y - half_window:y + half_window + 1, :]
            right_patch = right_image[x - half_window:x + half_window + 1, right_y - half_window:right_y + half_window + 1, :]

            cyclopean_patch = (left_patch + right_patch) // 2
            cyclopean_image[x - half_window:x + half_window + 1, y - half_window:y + half_window + 1, :] = cyclopean_patch

    cyclopean_image = np.clip(cyclopean_image, 0, 255).astype(np.uint8)
    return cyclopean_image

# Iterate over each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.bmp'):  # Check for .bmp extension
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Split the image into left and right views
        height, width, _ = image.shape
        mid = width // 2
        left_image = image[:, :mid, :]
        right_image = image[:, mid:, :]

        # Convert images to HSV
        left_image_hsv = cv2.cvtColor(left_image, cv2.COLOR_RGB2HSV)
        right_image_hsv = cv2.cvtColor(right_image, cv2.COLOR_RGB2HSV)

        # Compute gradients for left and right images
        grad_x_left, grad_y_left = compute_gradients(left_image_hsv)
        grad_x_right, grad_y_right = compute_gradients(right_image_hsv)

        # Compute z-gradient (difference between left and right images)
        diff_z = left_image_hsv.astype(np.float64) - right_image_hsv.astype(np.float64)

        # Initialize tensor components
        S11 = np.zeros(left_image_hsv.shape[:2])
        S12 = np.zeros(left_image_hsv.shape[:2])
        S13 = np.zeros(left_image_hsv.shape[:2])
        S22 = np.zeros(left_image_hsv.shape[:2])
        S23 = np.zeros(left_image_hsv.shape[:2])
        S33 = np.zeros(left_image_hsv.shape[:2])

        # Compute sums for each pixel across the 3 HSV channels
        for i in range(3):
            S11 += grad_x_left[:, :, i] ** 2
            S12 += grad_x_left[:, :, i] * grad_y_left[:, :, i]
            S13 += grad_x_left[:, :, i] * diff_z[:, :, i]
            S22 += grad_y_left[:, :, i] ** 2
            S23 += grad_y_left[:, :, i] * diff_z[:, :, i]
            S33 += diff_z[:, :, i] ** 2

        # Compute disparity map
        disparity_map = compute_disparity_map(S11, S12, S13, S22, S23, S33)

        # Create cyclopean image
        cyclopean_image = create_cyclopean_image(left_image_hsv, right_image_hsv, disparity_map)

        # Convert cyclopean image back to RGB
        cyclopean_image_rgb = cv2.cvtColor(cyclopean_image, cv2.COLOR_HSV2RGB)

        # Save the cyclopean image with the "_c" prefix
        output_filename = '_c' + filename
        output_path = os.path.join(output_folder, output_filename)
        cv2.imwrite(output_path, cyclopean_image_rgb)

        print(f'Saved cyclopean image for {filename} as {output_filename}')

print('Processing complete!')



Saved cyclopean image for 021image_5_0.bmp as _c021image_5_0.bmp
Saved cyclopean image for 013image_2_4.bmp as _c013image_2_4.bmp
Saved cyclopean image for 008image_4_2.bmp as _c008image_4_2.bmp
Saved cyclopean image for 010image_2_1.bmp as _c010image_2_1.bmp
Saved cyclopean image for 008image_1_0.bmp as _c008image_1_0.bmp
Saved cyclopean image for 009image_1_7.bmp as _c009image_1_7.bmp
Saved cyclopean image for 010image_1_3.bmp as _c010image_1_3.bmp
Saved cyclopean image for 021image_2_9.bmp as _c021image_2_9.bmp
Saved cyclopean image for 021image_1_4.bmp as _c021image_1_4.bmp
Saved cyclopean image for 007image_2_3.bmp as _c007image_2_3.bmp
Saved cyclopean image for 021image_3_0.bmp as _c021image_3_0.bmp
Saved cyclopean image for 008image_4_1.bmp as _c008image_4_1.bmp
Saved cyclopean image for 015image_2_6.bmp as _c015image_2_6.bmp
Saved cyclopean image for 007image_1_5.bmp as _c007image_1_5.bmp
Saved cyclopean image for 021image_5_3.bmp as _c021image_5_3.bmp
Saved cyclopean image for

In [ ]:
import shutil
import os

# Path to the folder containing cyclopean images
output_folder = '/content/Live Phase II/Cyclopean Image '

# Path where the zip file will be saved
zip_file_path = 'path_to_save_zip/cyclopean_images.zip'

# Compress the folder into a zip file
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', output_folder)

print(f'Folder compressed to: {zip_file_path}')


Folder compressed to: path_to_save_zip/cyclopean_images.zip


In [ ]:
from google.colab import files

# Path to the zip file you want to download
files.download(zip_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>